In [ ]:
import pandas as pd

for i in range(100,101):
    text_file_name = '40100' + str(i) + '.txt'
    output_file_name = 'merged_40100' + str(i) + '.xlsx'
    with open(text_file_name, 'r') as file:
        lines = file.readlines()

    data = {'SpO2': [], 'HR': [], 'PULSE': [], 'RESP': [], 'ABP1': [], 'ABP2': [], 'ABP3': [], 'INOP_PLETH': [], 'INOP_SpO2': [], 'INOP_RESP': [], 'INOP_NOISE':[],'ALARM':[]}

    for i in range (len(lines)):

        if lines[i]:
            Parts = lines[i].split()

            Key = Parts[0]
            Values = Parts[1:]
            # Check if the key is "RESP"
            if Key == "RESP":
                count=7
                pleth = False
                spo2=False
                resp = False
                noise =False
                alarm = False
                while True:
                    #write your code here to add the data into the dataframe from one "RESP" to next "RESP"
                    part1=lines[i].split()
                    key = part1[0]
                    values = part1[1:]
                    if key.startswith('ABP'):
                        data['ABP1'].append(values[0] if len(values) > 0 else None)
                        data['ABP2'].append(values[1] if len(values) > 1 else None)
                        data['ABP3'].append(values[2] if len(values) > 2 else None)
                    elif key.startswith('ALARM'):
                        data['ALARM'].append(values[0:] if len(values)> 0 else None)
                        alarm =True
                    elif key.startswith('INOP'):
                        if values[0] == 'PLETH':
                            data['INOP_PLETH'].append(' '.join(values[0:]) if len(values) > 1 else None)
                            pleth = True
                        elif values[0] =='SpO2':
                            data['INOP_SpO2'].append(' '.join(values[0:]) if len(values) > 1 else None)
                            spo2= True
                        elif values[0] == 'RESP':
                            data['INOP_RESP'].append(' '.join(values[0:]) if len(values) > 1 else None)
                            resp= True
                        elif values[0] != 'LEADS':
                            data['INOP_NOISE'].append(' '.join(values[0:]) if len(values) > 1 else None)
                            noise = True
                    else:
                        data[key].append(' '.join(values))

                    i = i+1
                    count =count -1
                    if i+1 < len(lines) :
                        next_part = lines[i].split()
                        next_key=next_part[0]
                        if next_key == "RESP":
                            break
                    else:
                        break
                if count <=2 :
                    if pleth==False :
                        data['INOP_PLETH'].append(0)
                    if spo2 == False:
                        data['INOP_SpO2'].append(0)
                    if resp == False:
                        data['INOP_RESP'].append(0)
                    if noise== False:
                        data['INOP_NOISE'].append(0)
                    if alarm == False:
                        data['ALARM'].append(0)

            else:
                continue



    # Fill missing values with None
    max_length = max(len(data[key]) for key in data)
    for key in data:
        data[key] += [None] * (max_length - len(data[key]))

    #converting text into integer

    columns = ['SpO2', 'HR', 'PULSE', 'RESP', 'ABP1', 'ABP2', 'ABP3']



    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    for column in columns:
        df[column]= pd.to_numeric(df[column],errors='coerce',downcast='integer')

    # Save the DataFrame to an Excel file
    df.to_excel(output_file_name, index=False)